# Visualization of electrostatics

In this lab, we will visualize the electrostatic potential from APBS.

#Part 0 – Downloading and Installing the required software

We will first install **py3Dmol** and **openbabel**. The latter will allow us to convert between file formats.

In [1]:
try:
  import py3Dmol
except:
  !pip install py3Dmol
  import py3Dmol

In [2]:
#Install conda using the conda-colab library
!pip install -q condacolab
import condacolab
condacolab.install_miniconda()

#Install openbabel
!conda install -c conda-forge openbabel --yes

✨🍰✨ Everything looks OK!
Solving environment: | / - \ | / - \ | / done

# All requested packages already installed.



Next, we will download and decompress the atomic coordinates and electrostatic potential into the hosted runtime.

In [4]:
!wget -L https://raw.githubusercontent.com/CCBatIIT/modelingworkshop/main/data/k5f10lg2t7_20220306/k5f10lg2t7.pqr
!wget -L https://raw.githubusercontent.com/CCBatIIT/modelingworkshop/main/data/k5f10lg2t7_20220306/k5f10lg2t7-pot.dx.gz
!gunzip k5f10lg2t7-pot.dx.gz

pqr = open('k5f10lg2t7.pqr', 'r').read()

wget: /usr/local/lib/libuuid.so.1: no version information available (required by wget)
--2022-03-07 18:21:43--  https://raw.githubusercontent.com/CCBatIIT/modelingworkshop/main/data/k5f10lg2t7_20220306/k5f10lg2t7.pqr
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 658081 (643K) [text/plain]
Saving to: ‘k5f10lg2t7.pqr’

k5f10lg2t7.pqr      100%[===================>] 642.66K  --.-KB/s    in 0.08s   

2022-03-07 18:21:43 (7.66 MB/s) - ‘k5f10lg2t7.pqr’ saved [658081/658081]

wget: /usr/local/lib/libuuid.so.1: no version information available (required by wget)
--2022-03-07 18:21:43--  https://raw.githubusercontent.com/CCBatIIT/modelingworkshop/main/data/k5f10lg2t7_20220306/k5f10lg2t7-pot.dx.gz
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 18

# Part I - Visualizing protonation

Let's look at all the histidines in the pqr file. What are the residue names?

In [5]:
for line in pqr.split('\n'):
  if line.find('HI')>-1 and line.find('CA')>-1:
    print(line)

ATOM    604  CA  HID A  41     -27.190  -2.710  28.609  0.0188 1.9080
ATOM    972  CA  HIE A  64     -39.219  -8.217  17.795 -0.0581 1.9080
ATOM   1221  CA  HID A  80     -33.509 -14.210  24.001  0.0188 1.9080
ATOM   2071  CA  HID A 134      -8.654  -5.431  35.198  0.0188 1.9080
ATOM   2489  CA  HID A 163     -16.507  -4.218  25.845  0.0188 1.9080
ATOM   2506  CA  HID A 164     -19.217  -2.931  28.181  0.0188 1.9080
ATOM   2633  CA  HIE A 172     -12.424   0.344  29.478 -0.0581 1.9080
ATOM   3789  CA  HIE A 246       5.892 -16.319  37.518 -0.0581 1.9080
ATOM   5304  CA  HID B  41      13.742 -12.686  -6.234  0.0188 1.9080
ATOM   5672  CA  HIE B  64       5.350 -25.189 -14.310 -0.0581 1.9080
ATOM   5921  CA  HID B  80       4.150 -15.056 -16.021  0.0188 1.9080
ATOM   6771  CA  HID B 134      12.044   4.775   3.073  0.0188 1.9080
ATOM   7189  CA  HID B 163       8.298  -6.805   1.500  0.0188 1.9080
ATOM   7206  CA  HID B 164      11.424  -7.630  -0.501  0.0188 1.9080
ATOM   7333  CA  HIE

Now look at the structure of a few histidines. The view below shows the protons on the histidine but not other protein aotms. Change the `resi` and `chain` variables below to zoom in to a different residue. Can you tell why a certain imidazole is stabilized?

In [7]:
resi = '163'
chain = 'A'

F = open('resi.pqr','w')
for line in pqr.split('\n'):
  if line[21:26] == f'{chain}{resi:>4s}':
    F.write(line + '\n')
F.close()

!obabel -ipqr resi.pqr -omol -O resi.mol 2> conversion.log
mol = open('resi.mol', 'r').read()

view = py3Dmol.view()
view.setBackgroundColor('white')
view.addModel(mol, 'mol')
view.setStyle({}, {'stick': {'colorscheme':'element'}})
view.addModel(pqr,'pdb')
sel = {'within':{'distance':'7', 'sel':{'and':[{'resi':resi}, {'chain':chain}]}}}
view.setStyle(sel, {'stick': {'colorscheme':'element'}})
view.zoomTo(sel)
view.show()

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

# Part II - Visualizing electrostatics

I was unable to get py3Dmol to show volumetric data in Google Colab. As an alternative, let's visualize the results with plot.ly. 

First, we will read the OpenDX file from APBS.

In [8]:
import numpy as np
F = open('k5f10lg2t7-pot.dx','r')

# Read the header
line = F.readline()
while line.find('object') == -1:
  line = F.readline()
header = {}
header['counts'] = [int(x) for x in line.split(' ')[-3:]]
for name in ['origin', 'd0', 'd1', 'd2']:
  header[name] = [float(x) for x in F.readline().split(' ')[-3:]]
F.readline()
header['npts'] = int(F.readline().split(' ')[-3])

# Read the data
vals = np.ndarray(shape=header['npts'], dtype=float)
index = 0
while index < header['npts']:
  line = F.readline()[:-1]
  items = [float(item) for item in line.split()]
  vals[index:index + len(items)] = items
  index = index + len(items)

F.close()


The file is too unwieldy to run on Google Colab. We will only use a subset of points

In [9]:
vals = vals.reshape(header['counts'])
vals = vals[::5,::5,::5]
print(vals.shape)

(39, 39, 39)


Now we will visualize a series of isosurfaes using plotly. Where is the electrostatic potential positive and negative?

In [10]:
def enable_plotly_in_cell(): # define once
  import IPython
  from plotly.offline import init_notebook_mode
  display(IPython.core.display.HTML('''<script src="/static/components/requirejs/require.js"></script>'''))
  init_notebook_mode(connected=False)
  
enable_plotly_in_cell() 
import plotly.graph_objects as go

X, Y, Z = np.mgrid[:vals.shape[0], :vals.shape[1], :vals.shape[2]]
X = X/header['d0'][0] - header['origin'][0]
Y = Y/header['d1'][1] - header['origin'][1]
Z = Z/header['d2'][2] - header['origin'][2]

fig = go.Figure(data=go.Volume(
    x=X.flatten(),
    y=Y.flatten(),
    z=Z.flatten(),
    value=vals.flatten(),
    isomin=-1,
    isomax=1,
    colorscale='BlueRed_r',
    opacity=0.1, # needs to be small to see through all surfaces
    surface_count=9, # needs to be a large number for good volume rendering
    ))
fig.show()

Output hidden; open in https://colab.research.google.com to view.